# 最小構成: Flask + Cloudflare Tunnel

たった2つのセルでAPIサーバーを公開します。

## セル1: インストール + Flask起動

In [ ]:
# インストール
!pip install -q flask flask-cors
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb

# Flask API
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading

app = Flask(__name__)
CORS(app)

@app.route("/api", methods=["POST"])
def api():
    data = request.json
    return jsonify({"message": "Hello from Colab!", "data": data})

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok"})

# バックグラウンドで起動
def run():
    app.run(host='0.0.0.0', port=8000)

threading.Thread(target=run, daemon=True).start()

print("✅ Flask起動完了 (port 8000)")
print("次のセルでCloudflare Tunnelを起動してください")

## セル2: Cloudflare Tunnel起動

In [ ]:
!cloudflared tunnel --url http://localhost:8000

## 使い方

1. **セル1を実行** → Flaskサーバーが起動
2. **セル2を実行** → Cloudflare URLが表示される

表示されるURL（例: `https://xxxx.trycloudflare.com`）をコピーして使用してください。

### テスト方法:

```bash
# ヘルスチェック
curl https://xxxx.trycloudflare.com/health

# APIテスト
curl -X POST https://xxxx.trycloudflare.com/api \
  -H 'Content-Type: application/json' \
  -d '{"name": "test"}'
```

### 出力例:

```
2024-01-15T12:34:56Z INF Thank you for trying Cloudflare Tunnel...
2024-01-15T12:34:56Z INF Your quick Tunnel has been created! Visit it at:
2024-01-15T12:34:56Z INF https://xxxx-yyyy-zzzz.trycloudflare.com
```

このURLがあなたのAPIの公開URLです！